---

# StemGNN Env Test

---

In [ ]:
!pip install numpy==1.19.5
!pip install scipy==1.5.4
!pip install pandas==1.1.5

In [4]:
import torch
print(torch.__version__, torch.cuda.is_available())
# 1.7.1+cu110 True

1.7.1 True


In [5]:
import os
#import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
from datetime import datetime
from models.handler import train, test

import argparse
import pandas as pd

parser = argparse.ArgumentParser()
parser.add_argument('--train', type=bool, default=True)
parser.add_argument('--evaluate', type=bool, default=True)
parser.add_argument('--dataset', type=str, default='ECG_data')
parser.add_argument('--window_size', type=int, default=12)
parser.add_argument('--horizon', type=int, default=3)
parser.add_argument('--train_length', type=float, default=7)
parser.add_argument('--valid_length', type=float, default=2)
parser.add_argument('--test_length', type=float, default=1)
parser.add_argument('--epoch', type=int, default=10)
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--multi_layer', type=int, default=5)
parser.add_argument('--device', type=str, default='cuda')
parser.add_argument('--validate_freq', type=int, default=1)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--norm_method', type=str, default='z_score')
parser.add_argument('--optimizer', type=str, default='RMSProp')
parser.add_argument('--early_stop', type=bool, default=False)
parser.add_argument('--exponential_decay_step', type=int, default=5)
parser.add_argument('--decay_rate', type=float, default=0.5)
parser.add_argument('--dropout_rate', type=float, default=0.5)
parser.add_argument('--leakyrelu_rate', type=int, default=0.2)


#args = parser.parse_args()
# print(f'Training configs: {args}')
args = parser.parse_args(args=[])

data_file = os.path.join('dataset', args.dataset + '.csv')
result_train_file = os.path.join('output', args.dataset, 'train')
result_test_file = os.path.join('output', args.dataset, 'test')
if not os.path.exists(result_train_file):
    os.makedirs(result_train_file)
if not os.path.exists(result_test_file):
    os.makedirs(result_test_file)
data = pd.read_csv(data_file).values

# split data
train_ratio = args.train_length / (args.train_length + args.valid_length + args.test_length)
valid_ratio = args.valid_length / (args.train_length + args.valid_length + args.test_length)
test_ratio = 1 - train_ratio - valid_ratio
train_data = data[:int(train_ratio * len(data))]
valid_data = data[int(train_ratio * len(data)):int((train_ratio + valid_ratio) * len(data))]
test_data = data[int((train_ratio + valid_ratio) * len(data)):]

In [7]:
torch.manual_seed(0)
if __name__ == '__main__':
    if args.train:
        try:
            before_train = datetime.now().timestamp()
            _, normalize_statistic = train(train_data, valid_data, args, result_train_file)
            after_train = datetime.now().timestamp()
            print(f'Training took {(after_train - before_train) / 60} minutes')
        except KeyboardInterrupt:
            print('Exiting from training early')
    if args.evaluate:
        before_evaluation = datetime.now().timestamp()
        test(test_data, args, result_train_file, result_test_file)
        after_evaluation = datetime.now().timestamp()
        print(f'Evaluation took {(after_evaluation - before_evaluation) / 60} minutes')
    print('done')

Total Trainable Params: 1123303
| end of epoch   0 | time:  4.29s | train_total_loss 0.8779
------ validate on data: VALIDATE ------
NORM: MAPE 95.271075898%; MAE 0.757578888; RMSE 1.002398337.
RAW : MAPE 97.260525566%; MAE 0.416039207; RMSE 0.704762356.
| end of epoch   1 | time:  3.51s | train_total_loss 0.8294
------ validate on data: VALIDATE ------
NORM: MAPE 94.529532073%; MAE 0.729549529; RMSE 0.973484193.
RAW : MAPE 94.679602967%; MAE 0.393543701; RMSE 0.661243011.
| end of epoch   2 | time:  4.92s | train_total_loss 0.8187
------ validate on data: VALIDATE ------
NORM: MAPE 94.141004882%; MAE 0.697229463; RMSE 0.946330694.
RAW : MAPE 92.227104106%; MAE 0.371006076; RMSE 0.627512707.
| end of epoch   3 | time:  4.06s | train_total_loss 0.8126
------ validate on data: VALIDATE ------
NORM: MAPE 94.012162080%; MAE 0.700190221; RMSE 0.946447276.
RAW : MAPE 91.969166444%; MAE 0.368673016; RMSE 0.615404751.
| end of epoch   4 | time:  4.31s | train_total_loss 0.8091
------ validate 